In [64]:
from collections import namedtuple, deque
import torch
import copy
class LSTMdata(object):
    def __init__(self, data):
        _, w, _ = data.shape
        self.__data = deque([], maxlen= w)
        self.__data_in_stock = deque([])
        self.init(data)

    def push(self, data):
        self.__data.append(data)
    
    def push_in_stock(self, data):
        self.__data_in_stock.append(data)

    def display(self):
        print(self.__data)

    def display_in_stock(self):
        print(self.__data_in_stock)   
    
    def init(self, data):
        l, w, _ = data.shape
        for i in range(w):
            self.push(data[0][i])
        for i in range(1, l):
            self.push_in_stock(data[i][-1])
    
    # 取得 RL 需要的當筆 data
    def load_RLdata(self):
        temp = self.__data[-1]
        print(temp)
        RLdata = torch.cat((temp[:2], temp[3:]))
        return(RLdata)
    
    # 將 RL 所生成的 data 放進 __data 變數
    def store_RLdata(self, RLdata):
       self.__data[-1][2] = RLdata

    # 取得 Time Series Model 需要的單筆 data
    def load_TSdata(self):
        print(list(self.__data))
        Tdata = torch.stack(list(self.__data))
        PDdata = copy.deepcopy(Tdata)
        PDdata[:, [0, 1]] = PDdata[:, [1, 0]]
        return(Tdata, PDdata)

    # 將 Time Series Model 所生成的 data 是下一個時間點的變化量。
    # 我們從 __data_in_stock 取得第一筆資料(即下一個時間點的變化量後)，並刪除 __data_in_stock該筆資料。
    # 我們將資料的第一筆和第二筆用模型預測出的溫度和工耗代替，然後放進 __data 變數。
    def store_TSdata(self, Tdata, PDdata):
        temp = self.__data_in_stock[0]
        temp[0] = Tdata
        temp[1] = PDdata
        self.push(temp)
        self.__data_in_stock.popleft()
        

In [65]:
import torch
l = 5
w = 3
h = 5
# 创建一个3x3x5的递增张量
t = torch.arange(l * w * h).reshape(l, w, h)

t

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]],

        [[30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44]],

        [[45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54],
         [55, 56, 57, 58, 59]],

        [[60, 61, 62, 63, 64],
         [65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74]]])

In [66]:
data = LSTMdata(t)
data.display()
print('-'*20)
data.display_in_stock()
data.load_RLdata()

deque([tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]), tensor([10, 11, 12, 13, 14])], maxlen=3)
--------------------
deque([tensor([25, 26, 27, 28, 29]), tensor([40, 41, 42, 43, 44]), tensor([55, 56, 57, 58, 59]), tensor([70, 71, 72, 73, 74])])
tensor([10, 11, 12, 13, 14])


tensor([10, 11, 13, 14])

In [67]:
data.store_RLdata(123)
data.display()

deque([tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]), tensor([ 10,  11, 123,  13,  14])], maxlen=3)


In [68]:
data.load_TSdata()

[tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]), tensor([ 10,  11, 123,  13,  14])]


(tensor([[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11, 123,  13,  14]]),
 tensor([[  1,   0,   2,   3,   4],
         [  6,   5,   7,   8,   9],
         [ 11,  10, 123,  13,  14]]))

In [69]:
data.display()
data.display_in_stock()
data.store_TSdata(1000, 2000)
data.display()
data.display_in_stock()

deque([tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]), tensor([ 10,  11, 123,  13,  14])], maxlen=3)
deque([tensor([25, 26, 27, 28, 29]), tensor([40, 41, 42, 43, 44]), tensor([55, 56, 57, 58, 59]), tensor([70, 71, 72, 73, 74])])
deque([tensor([5, 6, 7, 8, 9]), tensor([ 10,  11, 123,  13,  14]), tensor([1000, 2000,   27,   28,   29])], maxlen=3)
deque([tensor([40, 41, 42, 43, 44]), tensor([55, 56, 57, 58, 59]), tensor([70, 71, 72, 73, 74])])
